In [3]:
!apt-get update && apt-get install -y ffmpeg
!pip install moviepy openai-whisper detoxify transformers datasets

# Clone ParaDetox
!pip install datasets


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,268 kB]
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 1,653 kB in 1s (1,470 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu

In [4]:
!pip install -U datasets fsspec


  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)


In [3]:
from datasets import load_dataset
paradetox = load_dataset("s-nlp/paradetox")["train"]  # load dataset coc103
print(paradetox[0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'en_toxic_comment': 'he had steel balls too !', 'en_neutral_comment': 'he was brave too!'}


In [5]:
import whisper
model_script = whisper.load_model("large")

In [43]:
result = model_script.transcribe("/content/TheresNoOneAtAll.mp3")
text = result["text"]
print(text)

 I don't wanna make up Don't wanna make up, hands strong Don't wanna wake up so long I get it that hot and blue We seem that part of you I never knew love somehow Greasy cup that fumes Run away down that pain Raise a crown, unchain Closing my eyes, I hear your sigh I'm catching the light, stealing your wire Watching you lie, losing my mind Wishing you die, don't even try This a goodbye, ain't trying that deal We can't even drive, away from real Still skiing wide, wide, wide, wide, wide, wide There's no one out there There's no one out there There's no one out there There's no one out there There's no one out there There's no one out there You took me living nothing so did you feel the good same with some pretty Closing my eyes, eyeing your sire, catching the lines Stealing your wire, watching you lie Losing my mind, wishing you die Don't even try or this a goodbye Can't try and die to you, weaker than dry If you're with me or still skiing, why, why, why, why, why, why There's no one I 

In [6]:
result_2 = model_script.transcribe("/content/notlikeus.mp3")
text_2 = result_2["text"]
print(text_2)

 I Reincarnated I was stargazing life goes on honey. How my baby woke up looking for the broccoli Hockey keep a horn on me. That's a my seat. I'll be ownership. The blueprint is by me. Mr. Get off. I get off What's the password I see dead people You Lay No, they don't have no hype muster on the beat ho depot any rap nigga He'll freak though man down call it. Amber lambs. Tell him free bro. Nella nigga to the cross He walk around like T. So what's up with these? You're going to yes niggas trying to see Compton the industry can hate me fuck them all and mama how many I've she really got I mean It's too many options often the past on this body. I'm John Stockton Beat your ass and hot the Bible if God watching sometimes you gotta pop out and show niggas certified boogeyman I'm the one that up to score with him walking down whole time I know he got some horn him hold on him extortion bullied up throwing him say Drake I hear you like I'm young you better not have a go to cell block one To an

In [6]:
from detoxify import Detoxify
detector = Detoxify('original')
scores = detector.predict(text_2)
print(scores)  # xem điểm toxicity, obscene, insult...
# Phát hiện câu độc hại nếu scores['toxicity'] > 0.5


{'toxicity': np.float32(0.9702546), 'severe_toxicity': np.float32(0.28930634), 'obscene': np.float32(0.90278995), 'threat': np.float32(0.1259067), 'insult': np.float32(0.8044595), 'identity_attack': np.float32(0.7889489)}


In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Example thử inference
inp = "detoxify: " + "You're a fucking idiot."
ids = tokenizer(inp, return_tensors="pt").input_ids
out = model.generate(ids, max_length=32)
print(tokenizer.decode(out[0], skip_special_tokens=True))


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

: You're a fucking idiot.


In [7]:
from datasets import load_dataset
from transformers import T5Tokenizer

dataset = load_dataset("s-nlp/paradetox")["train"]
tokenizer = T5Tokenizer.from_pretrained("t5-base")

def preprocess(example):
    input_text = "detoxify: " + example["en_toxic_comment"]
    target_text = example["en_neutral_comment"]
    inputs = tokenizer(input_text, max_length=64, truncation=True, padding="max_length")
    targets = tokenizer(target_text, max_length=64, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized = dataset.map(preprocess, remove_columns=dataset.column_names)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/19744 [00:00<?, ? examples/s]

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create a DataLoader
# You can adjust batch_size as needed
batch_size = 8
dataloader = DataLoader(tokenized, batch_size=batch_size, collate_fn=data_collator)

# Example of how to iterate through a batch (optional)
for batch in dataloader:
    print({k: v.shape for k, v in batch.items()})
    break # Print only one batch for demonstration

In [15]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

model = T5ForConditionalGeneration.from_pretrained("t5-base")

training_args = TrainingArguments(
    output_dir="./t5-detox",  # Output directory for model checkpoints and logs
    per_device_train_batch_size=8,  # Batch size per GPU/TPU core/CPU for training
    num_train_epochs=3,  # Number of training epochs
    logging_steps=100,  # Log every N updates steps
    save_steps=500,  # Save checkpoint every N updates steps
    save_total_limit=2,  # Limit the total amount of checkpoints
    fp16=True,  # Use mixed precision training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: studentit1612 (studentit1612-uit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,2.046300
200,0.253300
300,0.224800
400,0.218600
500,0.220600
600,0.202900
700,0.215300
800,0.200700
900,0.200100
1000,0.197300


TrainOutput(global_step=7404, training_loss=0.20425547253660484, metrics={'train_runtime': 3148.0452, 'train_samples_per_second': 18.815, 'train_steps_per_second': 2.352, 'total_flos': 4508724221706240.0, 'train_loss': 0.20425547253660484, 'epoch': 3.0})

In [17]:
model.save_pretrained("./t5-detox-model")
tokenizer.save_pretrained("./t5-detox-model")


('./t5-detox-model/tokenizer_config.json',
 './t5-detox-model/special_tokens_map.json',
 './t5-detox-model/spiece.model',
 './t5-detox-model/added_tokens.json')

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("./t5-detox-model")
model = T5ForConditionalGeneration.from_pretrained("./t5-detox-model")

text_test = "detoxify: You are so damn annoying!"
input_ids = tokenizer(text_test, return_tensors="pt").input_ids
output_ids = model.generate(input_ids, max_length=64)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


You are so annoying


In [44]:
input_ids = tokenizer(text, return_tensors="pt").input_ids
output_ids = model.generate(input_ids, max_length=64)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

I don't wanna make up Don't wanna make up, hands strong Don't wanna wake up so long I get it that hot and blue We seem that part of you I never knew love somehow Raise a crown, unchain Closing my eyes, I hear your


In [48]:
text_test

'detoxify: You are so damn annoying!'

In [50]:
text

"detoxify:  I don't wanna make up Don't wanna make up, hands strong Don't wanna wake up so long I get it that hot and blue We seem that part of you I never knew love somehow Greasy cup that fumes Run away down that pain Raise a crown, unchain Closing my eyes, I hear your sigh I'm catching the light, stealing your wire Watching you lie, losing my mind Wishing you die, don't even try This a goodbye, ain't trying that deal We can't even drive, away from real Still skiing wide, wide, wide, wide, wide, wide There's no one out there There's no one out there There's no one out there There's no one out there There's no one out there There's no one out there You took me living nothing so did you feel the good same with some pretty Closing my eyes, eyeing your sire, catching the lines Stealing your wire, watching you lie Losing my mind, wishing you die Don't even try or this a goodbye Can't try and die to you, weaker than dry If you're with me or still skiing, why, why, why, why, why, why There'

In [51]:
input_ids = tokenizer(text, return_tensors="pt").input_ids
output_ids = model.generate(input_ids, max_length=10000)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

I don't wanna make up Don't wanna make up, hands strong Don't wanna wake up so long I get it that hot and blue We seem that part of you I never knew love somehow Raise a crown, unchain Closing my eyes, I hear your sigh I'm catching the light, stealing your wire Watching you lie, losing my mind Wishing you die, don't even try This a goodbye We can't even drive away from real Still skiing wide, wide


In [58]:
text_test = "detoxify: Hi good afternnon Hi How are you I'm fine Look What the fuck bro I don't know what the fucking shit"
input_ids = tokenizer(text_test, return_tensors="pt").input_ids
output_ids = model.generate(input_ids, max_length=64)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

Hi good afternnon Hi How are you I'm fine Look What bro I don't know what it is


In [9]:
!pip install nltk detoxify transformers

In [10]:
from detoxify import Detoxify
import torch
detox_model = Detoxify('original')  # model detect toxic

In [11]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Download the missing resource

from nltk.tokenize import sent_tokenize

from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("./t5-detox-model")
model = T5ForConditionalGeneration.from_pretrained("./t5-detox-model")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [12]:
def detoxify_paragraph(paragraph, prefix="detoxify: "):
    sentences = sent_tokenize(paragraph)
    detoxified_sentences = []

    for sent in sentences:
        input_text = prefix + sent
        input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).input_ids
        output_ids = model.generate(input_ids, max_length=64, do_sample=False)
        detox_sent = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        detoxified_sentences.append(detox_sent)

    return " ".join(detoxified_sentences)


In [13]:
def detoxify_paragraph_with_log(paragraph, toxic_threshold=0.5):
    sentences = sent_tokenize(paragraph)
    full_output = []

    for i, sent in enumerate(sentences):
        # Tính điểm toxicity
        scores = detox_model.predict(sent)
        toxic_score = scores["toxicity"]

        print(f"\n🟨 Câu {i+1}: {sent}")
        print(f"📊 Toxicity score: {toxic_score:.3f}")

        if toxic_score >= toxic_threshold:
            # Sinh câu sạch
            input_text = "detoxify: " + sent
            input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).input_ids
            with torch.no_grad():
                output_ids = model.generate(input_ids, max_length=64)
            detoxified = tokenizer.decode(output_ids[0], skip_special_tokens=True)

            print(f"✅ >> ĐÃ LÀM SẠCH: {detoxified}")
            full_output.append(detoxified)
        else:
            print("✔️ >> KHÔNG CẦN XỬ LÝ")
            full_output.append(sent)

    return " ".join(full_output)

In [63]:
long_text = """
You are so damn annoying! Shut the hell up and get lost. Why the f*** do you always act like a clown?
People like you should stay away from everyone. You're a total idiot and I can't stand your crap anymore!
"""

cleaned = detoxify_paragraph(long_text)
print("Cleaned Text:\n", cleaned)


Cleaned Text:
 You are so annoying Shut up and get lost. Why do you always act like a clown? People like you should stay away from everyone. I can't stand you anymore


In [65]:
text_2

" I Reincarnated I was stargazing life goes on honey. How my baby woke up looking for the broccoli Hockey keep a horn on me. That's a my seat. I'll be ownership. The blueprint is by me. Mr. Get off. I get off What's the password I see dead people You Lay No, they don't have no hype muster on the beat ho depot any rap nigga He'll freak though man down call it. Amber lambs. Tell him free bro. Nella nigga to the cross He walk around like T. So what's up with these? You're going to yes niggas trying to see Compton the industry can hate me fuck them all and mama how many I've she really got I mean It's too many options often the past on this body. I'm John Stockton Beat your ass and hot the Bible if God watching sometimes you gotta pop out and show niggas certified boogeyman I'm the one that up to score with him walking down whole time I know he got some horn him hold on him extortion bullied up throwing him say Drake I hear you like I'm young you better not have a go to cell block one To a

In [66]:
cleaned

"I Reincarnated I was stargazing life goes on honey. How my baby woke up looking for the broccoli Hockey keep a horn on me That's a my seat. I'll be ownership. The blueprint is by me. Mr. Get off I get off What's the password I see dead people You Lay No, they don't have any hype muster on the beat ho depot any rap nigga He'll freak though man down call it. Amber lambs. Tell him free bro. Nella to the cross He walk around like T. So what's up with these? You're going to yes people trying to see Compton the industry can hate me and mama how many I've she really got I mean It's too many options often the past on this body. I'm John Stockton I'm the one that up to score with him walking down whole time I know he got some horn him hold on him extortion bullied up throwing him say Drake I hear you like I'm young you better not have a go to cell block I'ma do much stuff White rolling like a naughty girl ain't you tired? I'm a strike a core and the cuts hate yeux They not like us they're not 

In [14]:
cleaned_2 = detoxify_paragraph_with_log(text_2)
print("\n\n🧼 FULL CLEANED TEXT:\n", cleaned_2)


🟨 Câu 1:  I Reincarnated I was stargazing life goes on honey.
📊 Toxicity score: 0.031
✔️ >> KHÔNG CẦN XỬ LÝ

🟨 Câu 2: How my baby woke up looking for the broccoli Hockey keep a horn on me.
📊 Toxicity score: 0.075
✔️ >> KHÔNG CẦN XỬ LÝ

🟨 Câu 3: That's a my seat.
📊 Toxicity score: 0.002
✔️ >> KHÔNG CẦN XỬ LÝ

🟨 Câu 4: I'll be ownership.
📊 Toxicity score: 0.001
✔️ >> KHÔNG CẦN XỬ LÝ

🟨 Câu 5: The blueprint is by me.
📊 Toxicity score: 0.001
✔️ >> KHÔNG CẦN XỬ LÝ

🟨 Câu 6: Mr. Get off.
📊 Toxicity score: 0.293
✔️ >> KHÔNG CẦN XỬ LÝ

🟨 Câu 7: I get off What's the password I see dead people You Lay No, they don't have no hype muster on the beat ho depot any rap nigga He'll freak though man down call it.
📊 Toxicity score: 0.843
✅ >> ĐÃ LÀM SẠCH: I get off What's the password I see dead people You Lay No, they don't have any hype muster on the beat ho depot any rap nigga He'll freak though man down call it.

🟨 Câu 8: Amber lambs.
📊 Toxicity score: 0.024
✔️ >> KHÔNG CẦN XỬ LÝ

🟨 Câu 9: Tell him

In [24]:
import os
import shutil

# Đường dẫn thư mục cần nén (ví dụ: "MyFolder" trong Drive)
folder_path = "/content/t5-detox-model"  # chỉnh sửa tên thư mục tại đây

# Tên file zip đầu ra
zip_path = "/content/"  # zip sẽ lưu ở /content (RAM Colab)

# Nén
shutil.make_archive(base_name=zip_path.replace(".zip", ""), format='zip', root_dir=folder_path)


'/content/.zip'

In [15]:
segments_2 = result_2["segments"]  # mỗi segment có: start, end, text


In [16]:
segments_2

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 2.0,
  'text': ' I',
  'tokens': [50365, 286, 50465],
  'temperature': 0.0,
  'avg_logprob': -0.4971392383314159,
  'compression_ratio': 1.4022988505747127,
  'no_speech_prob': 0.11551301181316376},
 {'id': 1,
  'seek': 0,
  'start': 6.24,
  'end': 12.8,
  'text': ' Reincarnated I was stargazing life goes on honey. How my baby woke up looking for the broccoli',
  'tokens': [50677,
   1300,
   41801,
   770,
   286,
   390,
   3543,
   70,
   2118,
   993,
   1709,
   322,
   8330,
   13,
   1012,
   452,
   3186,
   12852,
   493,
   1237,
   337,
   264,
   29044,
   51005],
  'temperature': 0.0,
  'avg_logprob': -0.4971392383314159,
  'compression_ratio': 1.4022988505747127,
  'no_speech_prob': 0.11551301181316376},
 {'id': 2,
  'seek': 0,
  'start': 12.88,
  'end': 19.080000000000002,
  'text': " Hockey keep a horn on me. That's a my seat. I'll be ownership. The blueprint is by me. Mr. Get off. I get off",
  'tokens': [51009,
   389,


In [17]:
def detoxify_t5(text):
    input_text = "detoxify: " + text.strip()
    input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).input_ids
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_length=64, do_sample=False)
    cleaned = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return cleaned

In [18]:
from detoxify import Detoxify

clean_segments = []
for seg in segments_2:
    score = Detoxify('original').predict(seg["text"])["toxicity"]
    if score >= 0.5:
        # Sửa bằng T5 hoặc GPT
        cleaned_text = detoxify_t5(seg["text"])  # bạn đã có hàm này rồi
        seg["cleaned"] = cleaned_text
        seg["need_replace"] = True
    else:
        seg["cleaned"] = seg["text"]
        seg["need_replace"] = False
    clean_segments.append(seg)


In [1]:
!pip install TTS


In [2]:
from TTS.api import TTS

# Load YourTTS (tiếng Anh multi-speaker + voice cloning)
tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=False, gpu=True)

# Voice clone từ file gốc .wav
reference_audio_path = "/content/notlikeus.mp3"


/usr/local/lib/python3.11/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--your_tts
 > Model's license - CC BY-NC-ND 4.0
 > Check https://creativecommons.org/licenses/by-nc-nd/4.0/ for more info.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 

In [19]:
from pathlib import Path
Path("cleaned_segments_audio").mkdir(exist_ok=True)

for idx, seg in enumerate(clean_segments):
    if seg["need_replace"]:
        output_path = f"cleaned_segments_audio/cleaned_{idx}.wav"
        tts.tts_to_file(
            text=seg["cleaned"],
            speaker_wav=reference_audio_path,
            file_path=output_path,
            language="en"  # YourTTS support multilingual, có thể để "en"
        )
        seg["cleaned_audio_path"] = output_path


 > Text splitted to sentences.
["No, they don't have any hype muster on the beat ho depot."]
 > Processing time: 6.404241561889648
 > Real-time factor: 1.4987693802690496
 > Text splitted to sentences.
["He'll freak though man down call it.", 'Amber lambs.', 'Tell him free bro.']
 > Processing time: 2.995145320892334
 > Real-time factor: 0.5029631101414499
 > Text splitted to sentences.
["You're going to yes guys trying to see Compton the industry can hate me"]
 > Processing time: 1.6791236400604248
 > Real-time factor: 0.4019927316400347
 > Text splitted to sentences.
['Beat yourself and the Bible if God watching sometimes you gotta pop out and show certified boogeyman']
 > Processing time: 3.7768895626068115
 > Real-time factor: 0.5351976140862705
 > Text splitted to sentences.
['To any girl that talk to him and they in love just make sure you hide your little sister from them']
 > Processing time: 1.7299504280090332
 > Real-time factor: 0.2875104583694587
 > Text splitted to sentenc

In [20]:
!pip install pydub
from pydub import AudioSegment


In [21]:
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [22]:
audio = AudioSegment.from_file("/content/notlikeus.mp3")


In [23]:
final_audio = AudioSegment.empty()
last_end_ms = 0

for idx, seg in enumerate(clean_segments):
    start_ms = int(seg["start"] * 1000)
    end_ms = int(seg["end"] * 1000)

    # Thêm phần giữa 2 đoạn
    final_audio += audio[last_end_ms:start_ms]

    if seg["need_replace"]:
        cleaned = AudioSegment.from_file(seg["cleaned_audio_path"])
        final_audio += cleaned
    else:
        final_audio += audio[start_ms:end_ms]

    last_end_ms = end_ms

# Thêm phần còn lại sau đoạn cuối cùng
final_audio += audio[last_end_ms:]


In [24]:
final_audio.export("audio_clean.wav", format="wav")


<_io.BufferedRandom name='audio_clean.wav'>